# ToDo List!
* ¿Qué suposición hace el MAE? ¿Qué está minimizando? ¿Por qué conviene usarlo como función de costo en este caso?
* Usar otros optimizadores

# Fuentes

### Link: https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0
En esta fuente se puede encontrar una breve explicación del MAE y del MSE, una comparación entre ambos respecto de su comportamiento en entrenamiento frente a conjuntos de datos con y sin outliers, y luego una comparación de su comportamiento durante entrenamiento a razón de cómo son sus gradientes, lo cual provoca en el caso del MAE que la convergencia sea más lenta y sea necesario utilizar un **learning rate dinámico**. Explica que, si nos importa que la presencia de outliers tenga un impacto directo sobre el modelo, deberíamos utilizar MSE, mientras que si deseamos que no afecte demasiado podemos emplear MAE.

### Link: https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
En esta fuente se puede encontrar una explicación de los tres métodos para learning rate dinámico utilizados, el **time-based decay**, el **step decay** y el **exponential decay**, empleando para algunos de ellos la clase de Keras llamada Learning Rate Scheduler, que permite modificar a gusto del usuario el valor del learning rate a través del proceso.

### Link: https://stackoverflow.com/questions/46308374/what-is-validation-data-used-for-in-a-keras-sequential-model
Esta disución de StackOverflow es interesante sobre la separación de los datasets en entrenamiento, validación y evaluación del modelo, la use para verificar algunas cuestiones sobre cómo usaba la información de validación Keras, entre otras cosas.

### Link: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
Explicación sobre el uso de **early stopping**, donde básicamente buscamos parar el entrenamiento aunque no se hayan terminado de correr todos los epochs predefinidos, porque se detecta que no hay mejoría en los resultados obtenidos, para ello se emplea la métrica evaluada sobre el conjunto de validación.

### Link: https://machinelearningmastery.com/polynomial-features-transforms-for-machine-learning/
Explicación sobre el uso de **features polinomiales**, que básicamente consiste en agregar nuevas variables de entrada al modelo a partir de potencias obtenidas entre las variables de entrada originales. De esta forma, el espacio que conforman las variables es de mayor dimensión y por ello la solución es más flexible, aunque hay que tener cuidado de que no se ajuste demasiado provocando **overfitting**.

# 1. Cargando base de datos

In [120]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [121]:
import numpy as np

In [122]:
import importlib

In [123]:
import sys

In [124]:
sys.path.insert(0, '../..')

In [125]:
# Read the database from the .csv file into a pandas dataframe
df = pd.read_csv('../../databases/insurance.csv')

# 2. Preprocesamiento de los datos

In [126]:
from sklearn import preprocessing

## 2.1. Codificación de variables no numéricas

In [127]:
# Create a label encoder for the sex variable or feature and create a new column in the dataframe 
# with the encoded version of the gender
sex_encoder = preprocessing.LabelEncoder()
sex_encoder.fit(df['sex'])
df['sex-encoded'] = sex_encoder.transform(df['sex'])

In [128]:
# Create a label encoder for the smoker variable or feature and create a new column in the dataframe
# with the encoded version of the smoker
smoker_encoder = preprocessing.LabelEncoder()
smoker_encoder.fit(df['smoker'])
df['smoker-encoded'] = smoker_encoder.transform(df['smoker'])

In [129]:
# Create a one hot encoder and fit the available types of regions in the dataset
region_encoder = preprocessing.OneHotEncoder()
region_encoder.fit(df['region'].to_numpy().reshape(-1, 1))

# Transform all entries into the one hot encoded representation
encoded_regions = region_encoder.transform(df['region'].to_numpy().reshape(-1, 1)).toarray()

# Add each new encoded variable or feature to the dataset
for i, category in enumerate(region_encoder.categories_[0]):
    df[f'{category}-encoded'] = encoded_regions.transpose()[i]

## 2.2. Filtrado de variables

In [130]:
# Filtering or removing of non desired variables
df_x = df[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded', 'northwest-encoded', 'northeast-encoded', 'southwest-encoded', 'southeast-encoded']]
df_y = df['charges']

# 3. Separación del conjunto de entrenamiento y evaluación

In [131]:
from sklearn import model_selection

In [132]:
from sklearn import preprocessing

## 3.1. Separación de los conjuntos
Es importante notar que, se realiza la separación del conjunto de datos original en **train**, **valid** y **test**, por fuera del framework de Keras para garantizar un adecuado tratamiento de los conjuntos acorde a la metodología empleada. En otras palabras, de esta forma nos aseguramos que cualquier preprocesamiento o normalización sobre validación (valid) y evaluación (test) se realiza a partir de la información obtenida en entrenamiento.

In [133]:
# Split the dataset into train_valid and test
x_train_valid, x_test, y_train_valid, y_test = model_selection.train_test_split(df_x, df_y, test_size=0.2, random_state=15, shuffle=True)

In [134]:
# Split the dataset into train and valid
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_train_valid, y_train_valid, test_size=0.2, random_state=23, shuffle=True)

# 4. Regresión Lineal


#### Comentarios
1. Al principio, sucedió que el MAE era muy lento para convergencia, lo cual tiene sentido por el tipo de función de costo que representa. Particularmente, comparado con MSE, es mucho más lentro. Empecé probando modificar de forma estática y a mano el **learning rate**.
2. Luego, con un learning rate cada vez mayor, pude observar que el entrenamiento era más rápido, pero sucedían dos cuestiones. En primer lugar, que se producía una especie oscilación en torno a un valor que asumo que es el mínimo al cual se acerca el entrenamiento, con lo cual sería necesario disminuir cerca de ahí el valor del learning rate. Por otro lado, este mínimo no era el mismo mínimo que obtuve con el MSE, debe ser un plateau, un mínimo local pero no el absoluto. Me propuse usar **learning rate dinámico** y **comenzar de diferentes puntos**.
3. Cuando probe utilizar MSE, si no normalizaba con z-score todas las variables, rápidamente divergía la función de costo y se rompía el entrenamiento. Por otro lado, la misma normalización afectaba mucho al entrenamiento del MAE. *¿Por qué?* Lo pude corregir un poco al aumentar el learning rate por un factor, lo cual debe tener sentido si se considera que ahora las variables estando normalizadas tienen una menor magnitud lo cual puede producir que los pasos sean menores que antes, y por eso se ralentizó.
4. Interesante, llegué a esta discusión https://datascience.stackexchange.com/questions/9020/do-i-have-to-standardize-my-new-polynomial-features a raiz de una pregunta bastante sencilla, **¿por qué no está mejorando la métrica con mayor orden de polynomial features?**. Resulta ser que normalizando las variables y luego aplicando polynomial features, obtengo nuevas variables que siguen encontrándose en el intervalo [0,1] pero que su orden de magnitud es mucho menor. *Conclusión, siempre normalizar las variables que entran al modelo, y por ende si aplicas polynomial features tenés que normalizar luego de crear las nuevas variables.*
5. Con la corrección mencionada anteriormente con respecto a la normalización, mejoró el resultado de ordenes grandes de polinomios.

In [175]:
from src import rl_helper
importlib.reload(rl_helper);

In [136]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=1000,
                          scheduler='time-decay',
                          decay_rate=0.01,
                          epochs=500,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-182113
Model checkpoints at checkpoints/rl/20210524-182113
Model: "sequential_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_47 (Dense)             (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 3151.51171875


In [137]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=10000,
                          scheduler='step-decay',
                          drop_rate=0.5,
                          epochs_drop=10,
                          epochs=500,
                          batch_size=32,
                         )

Model logs at tb-logs/rl/20210524-182204
Model checkpoints at checkpoints/rl/20210524-182204
Model: "sequential_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 3097.565673828125


In [138]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=10000,
                          scheduler='exponential-decay',
                          decay_rate=0.07,
                          epochs=500,
                          batch_size=32,
                         )

Model logs at tb-logs/rl/20210524-182307
Model checkpoints at checkpoints/rl/20210524-182307
Model: "sequential_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 3084.882568359375


In [139]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=1000,
                          scheduler='exponential-decay',
                          decay_rate=0.01,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=500,
                          batch_size=32,
                         )

Model logs at tb-logs/rl/20210524-182408
Model checkpoints at checkpoints/rl/20210524-182408
Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 3113.695068359375


In [141]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5000,
                          degree=2,
                          epochs=500,
                          batch_size=32,
                          scheduler='exponential-decay',
                          decay_rate=0.09
                         )

Model logs at tb-logs/rl/20210524-182601
Model checkpoints at checkpoints/rl/20210524-182601
Model: "sequential_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_51 (Dense)             (None, 1)                 55        
Total params: 55
Trainable params: 55
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 1708.3575439453125


In [143]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5000,
                          degree=3,
                          scheduler='exponential-decay',
                          decay_rate=0.1,
                          epochs=500,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-182835
Model checkpoints at checkpoints/rl/20210524-182835
Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_53 (Dense)             (None, 1)                 220       
Total params: 220
Trainable params: 220
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 1778.822265625


In [143]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5000,
                          degree=3,
                          scheduler='exponential-decay',
                          decay_rate=0.1,
                          epochs=500,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-182835
Model checkpoints at checkpoints/rl/20210524-182835
Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_53 (Dense)             (None, 1)                 220       
Total params: 220
Trainable params: 220
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 1778.822265625


In [144]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5000,
                          degree=3,
                          scheduler='exponential-decay',
                          decay_rate=0.1,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=500,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-182952
Model checkpoints at checkpoints/rl/20210524-182952
Model: "sequential_54"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 1)                 220       
Total params: 220
Trainable params: 220
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 1978.156494140625


In [145]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=1000,
                          degree=4,
                          scheduler='exponential-decay',
                          decay_rate=0.1,
                          epochs=500,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-183059
Model checkpoints at checkpoints/rl/20210524-183059
Model: "sequential_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 1)                 715       
Total params: 715
Trainable params: 715
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 1885.450927734375


In [146]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=1000,
                          degree=4,
                          scheduler='exponential-decay',
                          decay_rate=0.1,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=500,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-183215
Model checkpoints at checkpoints/rl/20210524-183215
Model: "sequential_56"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 1)                 715       
Total params: 715
Trainable params: 715
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 2000.34423828125


In [158]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=2.0,
                          degree=5,
                          scheduler='exponential-decay',
                          decay_rate=0.001,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=1000,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-195556
Model checkpoints at checkpoints/rl/20210524-195556
Model: "sequential_67"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 1)                 2002      
Total params: 2,002
Trainable params: 2,002
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 1855.770751953125


In [152]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5.0,
                          degree=6,
                          scheduler='exponential-decay',
                          decay_rate=0.01,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=1000,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-190323
Model checkpoints at checkpoints/rl/20210524-190323
Model: "sequential_61"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 1)                 5005      
Total params: 5,005
Trainable params: 5,005
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 1921.7529296875


In [153]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5.0,
                          degree=7,
                          scheduler='exponential-decay',
                          decay_rate=0.01,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=1000,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-190557
Model checkpoints at checkpoints/rl/20210524-190557
Model: "sequential_62"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_62 (Dense)             (None, 1)                 11440     
Total params: 11,440
Trainable params: 11,440
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 1921.1785888671875


In [154]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5.0,
                          degree=8,
                          scheduler='exponential-decay',
                          decay_rate=0.01,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=1000,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-190913
Model checkpoints at checkpoints/rl/20210524-190913
Model: "sequential_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_63 (Dense)             (None, 1)                 24310     
Total params: 24,310
Trainable params: 24,310
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 2113.119384765625


In [172]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5.0,
                          degree=9,
                          scheduler='exponential-decay',
                          decay_rate=0.01,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=1000,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-201747
Model checkpoints at checkpoints/rl/20210524-201747
Model: "sequential_78"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_78 (Dense)             (None, 1)                 48620     
Total params: 48,620
Trainable params: 48,620
Non-trainable params: 0
_________________________________________________________________


KeyboardInterrupt: 